In [1]:
import spacy
nlp = spacy.load("en_core_web_trf")

In [2]:
import os
files = os.listdir("C:/Users/SRINI/RECAM/Input")
files

['Untitled - 0600.txt', 'Untitled - 0601.txt']

In [3]:
from pathlib import Path
import colored
test = 0
text = ""
for file in files:
    with open(os.path.join("C:/Users/SRINI/RECAM/Input", file), "r", encoding="utf8") as f:
        if len(text) <= 950000:
            file_path = os.path.join("C:/Users/SRINI/RECAM/Input", file)
            text = text + "\n\n\n" + file_path + "\n\n\n" + f.read()
            test = test + 1
            print(test)
        else:
            break
    doc = nlp(text)
    # Do something with the doc object

1
2


In [4]:
#Import the requisite library
# import spacy

#Build upon the spaCy Small Model
# nlp = spacy.load("en_core_web_sm")

#Sample text
# text = "Fever past 10 days can we take Dolo medicine"

corpus = []

# doc = nlp(text)
for sent in doc.sents:
    corpus.append(sent.text)

#Build upon the spaCy Small Model
nlp = spacy.blank("en")

#Create the EntityRuler
ruler = nlp.add_pipe("entity_ruler")

#List of Entities and Patterns
patterns = [
                {"label": "Alterations1", "pattern": [{"LOWER": "alterations"}]}, 
                {"label": "Assignment1", "pattern": [{"LOWER": {"IN": ["assignment"]}}]}, 
                {"label": "Audit1", "pattern": [{"LOWER": {"IN": ["audit"]}}]}, 
                {"label": "Casualty1", "pattern": [{"LOWER": {"IN": ["casualty"]}}]}, 
                {"label": "Default1", "pattern": [{"LOWER": {"IN": ["default"]}}]}, 
                {"label": "Condemnation1", "pattern": [{"LOWER": {"IN": ["condemnation"]}}]}, 
                {"label": "Eminent1", "pattern": [{"LOWER": {"IN": ["eminent"]}}]}, 
                {"label": "Within1", "pattern": [{"LOWER": {"IN": ["within"]}}]}, 
                {"label": "Hazardous1", "pattern": [{"LOWER": {"IN": ["hazardous"]}}]}, 
                {"label": "Terminable1", "pattern": [{"LOWER": {"IN": ["terminable"]}}]}, 
                {"label": "HVAC1", "pattern": [{"LOWER": {"IN": ["hvac"]}}]}, 
                {"label": "Insurance1", "pattern": [{"LOWER": {"IN": ["insurance"]}}]}, 
                {"label": "Late1", "pattern": [{'LOWER': {'IN': ["late"]}}]}, 
                {"label": "Notices1", "pattern": [{'LOWER': {'IN': ["notices"]}}]}, 
                {"label": "Space1", "pattern": [{"LOWER": {"IN": ["space"]}}]}, 
                {"label": "Subordinate1", "pattern": [{"LOWER": {"IN": ["subordinate"]}}]}, 
                {"label": "Signage1", "pattern": [{"LOWER": {"IN": ["signage"]}}]}, 
                {"label": "RET1", "pattern": [{"LOWER": {"IN": ["ret"]}}]}, 
                {"label": "CPI1", "pattern": [{"LOWER": {"IN": ["cpi"]}}]}
                #{"label": "PHONE_NUMBER", "pattern": [{"ORTH": "("}, {"SHAPE": "ddd"}, {"ORTH": ")"}, {"SHAPE": "ddd"}, {"ORTH": "-", "OP": "?"}, {"SHAPE": "dddd"}]}, 
                #{"label": "Abstract_Keywords", "pattern": [{'LOWER': {'IN': ['landlord', 'tenant', 'effective date', 'commencement date', 'base rent']}}]}, 
                #{"label": "Effective", "pattern": "Effective Date"}, 
                #{"label": "Commencement", "pattern": "Commencement Date"}, 
                #{"label": "Base Rent", "pattern": "Base Rent"}, 
                #{"label": "Renewal", "pattern": "Renewal"}, 
                #{"label": "MONEY", "pattern": [{"ORTH": "$"}, {"LIKE_NUM": True}]}
                #{"label": "DATE_FORMAT", "pattern": [{"LOWER": {"REGEX": "(January|February|March|April|May|June|July|August|September|October|November|December)( ) (\d){1,2}(\, ) (\d){1,4}"}}]}, 
                #{"label": "DATE_FORMAT", "pattern": [{"POS": "PROPN"}, {"LIKE_NUM": True}, {"IS_PUNCT": True}, {"LIKE_NUM": True}]}, 
                #{"label": "DATE_FORMAT", "pattern": [{"POS": "PROPN"}, {"LIKE_NUM": True}, {"text": ","}, {"LIKE_NUM": True}]}, 
                #{"label": "DATE_FORMAT", "pattern": [{"LIKE_DATE": True}]}, 
                #{"label": "MONEY", "pattern": "$"}
            ]

#Create the EntityRuler
#ruler = nlp.add_pipe("entity_ruler", before="ner")
#ruler = nlp.add_pipe("entity_ruler", before="ner")

ruler.add_patterns(patterns)


TRAIN_DATA = []

#iterate over the corpus again
for sentence in corpus:
    doc = nlp(sentence)
    
    #remember, entities needs to be a dictionary in index 1 of the list, so it needs to be an empty list
    entities = []
    
    #extract entities
    for ent in doc.ents:

        #appending to entities in the correct format
        entities.append([ent.start_char, ent.end_char, ent.label_])
        
       
    TRAIN_DATA.append([sentence, {"entities": entities}])
    
 
print (TRAIN_DATA)

[['\n\n\nC:/Users/SRINI/RECAM/Input\\Untitled - 0600.txt\n\n\n\ufeffshall have accrued to, or with respect to, any period ending at the time of expiration or other termination of this Lease shall survive the\nExpiration Date or other termination of this Lease.\n26.7\tProrations.', {'entities': []}], ['Any apportionments or prorations of Rent to be made under this Lease shall be computed on the basis of a year containing three hundred sixty (360) days, consisting of twelve (12) months of thirty (30) days each.', {'entities': []}], ['\n26.8\tGoverning Law; Construction.', {'entities': []}], ['This Lease shall be governed by and construed in accordance with the laws of the state in which the Project is located.', {'entities': []}], ['If any provision of this Lease or the application thereof to any person or circumstance shall, for any reason and to any extent, be invalid or unenforceable, the remainder of this Lease and the application of that provision to other persons or circumstances s

In [5]:
import srsly
import typer
import warnings
from pathlib import Path

import spacy
from spacy.tokens import DocBin

def convert(lang: str, TRAIN_DATA, output_path: Path):
    nlp = spacy.blank(lang)
    db = DocBin()
    for text, annot in TRAIN_DATA:
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label)
            if span is None:
                msg = f"Skipping entity [{start}, {end}, {label}] in the following text because the character span '{doc.text[start:end]}' does not align with token boundaries:\n\n{repr(text)}\n"
                warnings.warn(msg)
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    db.to_disk(output_path)

In [6]:
convert("en", TRAIN_DATA, "data/train.spacy")
convert("en", TRAIN_DATA, "data/valid.spacy")

In [7]:
!python -m spacy init fill-config data/base_config.cfg data/config.cfg

[+] Auto-filled config with all values
[+] Saved config
data\config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [9]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
!python -m spacy train data/config.cfg --output ./models/output --paths.train ./data/train.spacy --paths.dev ./data/valid.spacy

[i] Saving to output directory: models\output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     40.90    0.00    0.00    0.00    0.00
  7     200         16.58    764.41   97.14   97.14   97.14    0.97
 15     400          0.72      8.42  100.00  100.00  100.00    1.00
 26     600          0.00      0.00  100.00  100.00  100.00    1.00
 40     800          0.00      0.00  100.00  100.00  100.00    1.00
 57    1000          0.00      0.00  100.00  100.00  100.00    1.00
 78    1200          0.00      0.00  100.00  100.00  100.00    1.00
105    1400          0.00      0.00  100.00  100.00  100.00    1.00
138    1600   

In [11]:
import spacy
from spacy.matcher import Matcher
import tkinter as tk
from tkinter import filedialog
import os
from pathlib import Path

from spacy import displacy
# import displacy
from io import BytesIO
from pdfkit import from_string
import pdfkit


nlp = spacy.load("en_core_web_trf")

# Function to update the Listbox with folder contents
def browse_folder():
    global text1, file_path
    text1 = ""
    file_count = 0
    folder_path = filedialog.askdirectory()  # Open a folder selection dialog
    if folder_path:
        folder_contents.delete(0, tk.END)  # Clear the Listbox
        for item in os.listdir(folder_path):
            if item.endswith(".txt"):
                folder_contents.insert(tk.END, item)  # Insert folder contents into Listbox
                file_path = folder_path
                with open(os.path.join(file_path, item), "r", encoding="utf8") as f:
                    if len(text1) <= 950000:
                        text1 = text1 + "\n\n" + file_path + "\n\n" + "=======\n\n" + item + "\n\n=======\n\n" + f.read()
                        file_count = file_count + 1
                        print(file_count)
                    else:
                        break
                doc = nlp(text1)
                #doc1 = nlp(text1)
                
                
                # print(get_amount_headline(doc))    
                # print(get_date(doc)) 
                
                # print(get_pay_date(doc))
                # print(get_amount_summary(doc))
    
    html = displacy.render(doc, style="ent", page=True, jupyter=False)
    config = pdfkit.configuration(wkhtmltopdf="C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe")
    print(file_path)
    pdfkit.from_string(html, file_path + "/" + 'Automation.pdf', configuration=config)
    # displacy.render(doc, style="ent", page=True, jupyter=True)
    
    trained_nlp = spacy.load("models/output/model-best")
    #text = "Fever past 10 days can we take Dolo medicine, Dolo used for all body pain not only for Fever, so we can use this Dolo for many aspect"
    doc = trained_nlp(text1)
    for ent in doc.ents:
        print (ent.text, ent.label_)
    html = displacy.render(doc, style="ent", page=True, jupyter=False)
    config = pdfkit.configuration(wkhtmltopdf="C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe")
    pdfkit.from_string(html, file_path + "/" + 'LeaseAbs.pdf', configuration=config)
    
    # trained_nlp = spacy.load("models/output/model-best")
    #text = "Fever past 10 days can we take Dolo medicine, Dolo used for all body pain not only for Fever, so we can use this Dolo for many aspect"
    #doc1 = trained_nlp(text1)
    #for ent1 in doc1.ents:
     #   if ent1.label_ == "MONEY":
      #      print(ent1.text, ent1.label_)
       #     print("Inside MONEY")
    #for ent in doc.ents:
    #    print (ent.text, ent.label_)
    #html = displacy.render(doc1, style="ent", page=True, jupyter=False)
    #config = pdfkit.configuration(wkhtmltopdf="C:\\Program Files\\wkhtmltopdf\\bin\\wkhtmltopdf.exe")
    #pdfkit.from_string(html, file_path + "/" + 'Date.pdf', configuration=config)
    #doc1
    # get_date(doc)
    # displacy.render(doc, style="ent", page=True, jupyter=True)
    # print(get_amount_headline(doc))    
    # print(get_date(doc)) 
    # print(get_pay_date(doc))
    # print(get_amount_summary(doc))
    
from tkinter import *

# Create the main window
root = tk.Tk()

#getting screen width and height of display
width= root.winfo_screenwidth() 
height= root.winfo_screenheight()
#setting tkinter window size
root.geometry("%dx%d" % (width, height))

# root.title("RECAM AIML LEASE ABSTRACTION PREDICTION")
root.title("Lease Abstraction Private Limited")

title = tk.Label(root, text="AIML LEASE ABSTRACTION PREDICTION")
title.pack()

# Change the font size of the title
title.config(font=("Helvetica", 30))
title.config(fg="lemonchiffon")
title.config(bg="chocolate")

# Create a button to browse for a folder
browse_button = tk.Button(root, text="Select Text File Folder", command=browse_folder)
browse_button.pack(pady=10)
browse_button.config(font=("Courier", 20))
browse_button.config(fg="forestgreen")
browse_button.config(bg="floralwhite")
    
# Create a Listbox to display folder contents
folder_contents = tk.Listbox(root, selectmode=tk.SINGLE)
folder_contents.pack(fill=tk.BOTH, expand=True)
folder_contents.config(font=("Courier", 20))
folder_contents.config(fg="ghostwhite")
folder_contents.config(bg="gray")

# Create a scrollbar for the Listbox
scrollbar = tk.Scrollbar(folder_contents, orient=tk.VERTICAL)
scrollbar.config(command=folder_contents.yview)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
folder_contents.config(yscrollcommand=scrollbar.set)

# Start the Tkinter main loop
root.mainloop()

1
2
C:/Users/SRINI/RECAM/Input
Default Default1
within Within1
space Space1
assignment Assignment1
HVAC HVAC1
HVAC HVAC1
HVAC HVAC1
Space Space1
Space Space1
HVAC HVAC1
HVAC HVAC1
HVAC HVAC1
HVAC HVAC1
HVAC HVAC1
notices Notices1
HVAC HVAC1
HVAC HVAC1
HVAC HVAC1
HVAC HVAC1
SPACE Space1
SPACE Space1
SPACE Space1
HVAC HVAC1
HVAC HVAC1
HVAC HVAC1
within Within1
HVAC HVAC1
HVAC HVAC1
HVAC HVAC1
default Default1
HVAC HVAC1
insurance Insurance1
HVAC HVAC1
HVAC HVAC1
HVAC HVAC1
